# Oracle AI Vector Search: 生成嵌入
Oracle AI Vector Search 专为人工智能 (AI) 工作负载而设计，允许您根据语义而非关键词查询数据。
Oracle AI Vector Search 的最大优势之一在于，可以将非结构化数据的语义搜索与业务数据的关系搜索结合在同一个系统中。
这不仅功能强大，而且效果显著，因为您无需添加专门的向量数据库，从而消除了多系统间数据碎片化的烦恼。

此外，您的向量还可以受益于 Oracle Database 所有最强大的功能，例如：

 * [分区支持](https://www.oracle.com/database/technologies/partitioning.html)
 * [Real Application Clusters 可扩展性](https://www.oracle.com/database/real-application-clusters/)
 * [Exadata 智能扫描](https://www.oracle.com/database/technologies/exadata/software/smartscan/)
 * [跨地理分布式数据库的分片处理](https://www.oracle.com/database/distributed-database/)
 * [事务](https://docs.oracle.com/en/database/oracle/oracle-database/23/cncpt/transactions.html)
 * [并行 SQL](https://docs.oracle.com/en/database/oracle/oracle-database/21/vldbg/parallel-exec-intro.html#GUID-D28717E4-0F77-44F5-BB4E-234C31D4E4BA)
 * [灾难恢复](https://www.oracle.com/database/data-guard/)
 * [安全性](https://www.oracle.com/security/database-security/)
 * [Oracle 机器学习](https://www.oracle.com/artificial-intelligence/database-machine-learning/)
 * [Oracle 图数据库](https://www.oracle.com/database/integrated-graph-database/)
 * [Oracle Spatial and Graph](https://www.oracle.com/database/spatial/)
 * [Oracle 区块链](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_blockchain_table.html#GUID-B469E277-978E-4378-A8C1-26D3FF96C9A6)
 * [JSON](https://docs.oracle.com/en/database/oracle/oracle-database/23/adjsn/json-in-oracle-database.html)


本指南演示了如何在 Oracle AI Vector Search 中使用嵌入功能，通过 OracleEmbeddings 为您的文档生成嵌入。

如果您刚开始接触 Oracle Database，可以考虑探索 [免费的 Oracle 23 AI](https://www.oracle.com/database/free/#resources)，它提供了一个很好的入门环境来设置您的数据库环境。在处理数据库时，通常建议不要默认使用 system 用户；您可以创建自己的用户以增强安全性和自定义性。有关用户创建的详细步骤，请参阅我们的 [端到端指南](https://github.com/langchain-ai/langchain/blob/master/cookbook/oracleai_demo.ipynb)，其中还演示了如何在 Oracle 中设置用户。此外，了解用户权限对于有效管理数据库安全至关重要。您可以在官方的 [Oracle 指南](https://docs.oracle.com/en/database/oracle/oracle-database/19/admqs/administering-user-accounts-and-security.html#GUID-36B21D72-1BBB-46C9-A0C9-F0D2A8591B8D) 中了解有关用户账户和安全管理方面的更多信息。

### 先决条件

请确保已安装 Oracle Python 客户端驱动程序，以便将 Langchain 与 Oracle AI Vector Search 进行集成。

In [ ]:
# pip install oracledb

### 连接到 Oracle 数据库
下面的示例代码将展示如何连接到 Oracle 数据库。默认情况下，python-oracledb 以“Thin”模式运行，该模式直接连接到 Oracle 数据库。此模式不需要 Oracle Client 库。但是，当 python-oracledb 使用 Oracle Client 库时，可以使用一些附加功能。当使用 Oracle Client 库时，python-oracledb 被称为处于“Thick”模式。两种模式都支持全面的功能，符合 Python Database API v2.0 规范。请参阅以下 [指南](https://python-oracledb.readthedocs.io/en/latest/user_guide/appendix_a.html#featuresummary)，其中介绍了每种模式支持的功能。如果无法使用 Thin 模式，您可能需要切换到 Thick 模式。

In [ ]:
import sys

import oracledb

# Update the following variables with your Oracle database credentials and connection details
username = "<username>"
password = "<password>"
dsn = "<hostname>/<service_name>"

try:
    conn = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")
    sys.exit(1)

在嵌入生成方面，用户可以选择多种提供程序，包括数据库内的嵌入生成以及 OcigenAI、Hugging Face 和 OpenAI 等第三方服务。选择第三方提供程序的用户的凭据必须包含必需的身份验证信息。或者，如果用户选择“数据库”作为其提供程序，则需要将 ONNX 模型加载到 Oracle Database 中以方便生成嵌入。

### 加载 ONNX 模型

Oracle 支持多种嵌入模型提供商，使用户可以在专有的数据库解决方案以及 OCIGENAI 和 HuggingFace 等第三方服务之间进行选择。此选择决定了生成和管理嵌入模型的方法。

***重要***：如果用户选择数据库选项，则必须将 ONNX 模型上传到 Oracle 数据库。反之，如果选择第三方提供商进行嵌入模型生成，则无需将 ONNX 模型上传到 Oracle 数据库。

直接在 Oracle 中使用 ONNX 模型的一个显著优势是它具有更高的安全性和性能，消除了将数据传输给外部方的需求。此外，此方法还避免了通常与网络或 REST API 调用相关的延迟。

以下是将 ONNX 模型上传到 Oracle 数据库的示例代码：

In [ ]:
from langchain_community.embeddings.oracleai import OracleEmbeddings

# Update the directory and file names for your ONNX model
# make sure that you have onnx file in the system
onnx_dir = "DEMO_DIR"
onnx_file = "tinybert.onnx"
model_name = "demo_model"

try:
    OracleEmbeddings.load_onnx_model(conn, onnx_dir, onnx_file, model_name)
    print("ONNX model loaded.")
except Exception as e:
    print("ONNX model loading failed!")
    sys.exit(1)

### 创建凭证

在选择第三方提供商生成嵌入时，用户需要建立凭证以安全地访问提供商的端点。

***重要提示：*** 当选择“数据库”提供商生成嵌入时，不需要任何凭证。但是，如果用户决定使用第三方提供商，则必须创建特定于所选提供商的凭证。

下面是一个示例：

In [ ]:
try:
    cursor = conn.cursor()
    cursor.execute(
        """
       declare
           jo json_object_t;
       begin
           -- HuggingFace
           dbms_vector_chain.drop_credential(credential_name  => 'HF_CRED');
           jo := json_object_t();
           jo.put('access_token', '<access_token>');
           dbms_vector_chain.create_credential(
               credential_name   =>  'HF_CRED',
               params            => json(jo.to_string));

           -- OCIGENAI
           dbms_vector_chain.drop_credential(credential_name  => 'OCI_CRED');
           jo := json_object_t();
           jo.put('user_ocid','<user_ocid>');
           jo.put('tenancy_ocid','<tenancy_ocid>');
           jo.put('compartment_ocid','<compartment_ocid>');
           jo.put('private_key','<private_key>');
           jo.put('fingerprint','<fingerprint>');
           dbms_vector_chain.create_credential(
               credential_name   => 'OCI_CRED',
               params            => json(jo.to_string));
       end;
       """
    )
    cursor.close()
    print("Credentials created.")
except Exception as ex:
    cursor.close()
    raise

### 生成嵌入

Oracle AI Vector Search 提供多种生成嵌入的方法，可利用本地托管的 ONNX 模型或第三方 API。有关配置这些替代方案的全面说明，请参阅 [Oracle AI Vector Search 指南](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_vector_chain1.html#GUID-C6439E94-4E86-4ECD-954E-4B73D53579DE)。

***注意：*** 用户可能需要配置代理才能利用第三方嵌入生成提供商，但不包括使用 ONNX 模型的 'database' 提供商。

In [ ]:
# proxy to be used when we instantiate summary and embedder object
proxy = "<proxy>"

以下示例代码将展示如何生成 embeddings：

In [ ]:
from langchain_community.embeddings.oracleai import OracleEmbeddings
from langchain_core.documents import Document

"""
# using ocigenai
embedder_params = {
    "provider": "ocigenai",
    "credential_name": "OCI_CRED",
    "url": "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com/20231130/actions/embedText",
    "model": "cohere.embed-english-light-v3.0",
}

# using huggingface
embedder_params = {
    "provider": "huggingface", 
    "credential_name": "HF_CRED", 
    "url": "https://api-inference.huggingface.co/pipeline/feature-extraction/", 
    "model": "sentence-transformers/all-MiniLM-L6-v2", 
    "wait_for_model": "true"
}
"""

# using ONNX model loaded to Oracle Database
embedder_params = {"provider": "database", "model": "demo_model"}

# If a proxy is not required for your environment, you can omit the 'proxy' parameter below
embedder = OracleEmbeddings(conn=conn, params=embedder_params, proxy=proxy)
embed = embedder.embed_query("Hello World!")

""" verify """
print(f"Embedding generated by OracleEmbeddings: {embed}")

### 端到端演示

请参考我们完整的演示指南 [Oracle AI Vector Search 端到端演示指南](https://github.com/langchain-ai/langchain/tree/master/cookbook/oracleai_demo.ipynb)，借助 Oracle AI Vector Search 构建端到端的 RAG 管道。